In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

In [ ]:
# Import the required libraries
import os

# Define the location of the directory
path =r"D:\data\Work project\Check Plagiarism\Dataset\Figure Plagiarism corpus\Shape_Based_Figure Plagiarism\Source figures"

# Change the directory
os.chdir(path)



In [ ]:
import string
def remove_punctuations(txt, punct = string.punctuation):
    '''
    This function will remove punctuations from the input text
    '''
    return ''.join([c for c in txt if c not in punct])
  
def remove_stopwords(txt, sw = list(stopwords.words('english'))):
    '''
    This function will remove the stopwords from the input txt
    '''
    return ' '.join([w for w in txt.split() if w.lower() not in sw])

def clean_text(txt):
    '''
    This function will clean the text being passed by removing specific line feed characters
    like '\n', '\r', and '\'
    '''
    
    txt = txt.replace('\n', ' ').replace('\r', ' ').replace('\'', '')
    txt = remove_punctuations(txt)
    txt = remove_stopwords(txt)
    return txt.lower()

In [ ]:
def read_files(file_path):
    result = []
    with open(file_path, encoding="cp437") as file:
        for line in file.readlines():
            a = line.find("coordinate")
            line = line[:a]
            line = clean_text(line)
            line = line.replace("type","")
            line = line.replace("text","")
            line = line.lower()
            result.append(line)
    return result

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Build training data

In [ ]:
train_files = [doc for doc in os.listdir() if doc.endswith('.txt')]

In [ ]:
#train_files

In [ ]:
OLD_PATH = "D:\data\Work project\Check Plagiarism\Dataset\Figure Plagiarism corpus\Shape_Based_Figure Plagiarism\Source figures/"

train_contents = [read_files(os.path.join(OLD_PATH,File)) for File in train_files]

In [ ]:
read_files(os.path.join(OLD_PATH,"Source-figure-100_out.txt")) 

In [ ]:
#train_contents

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(2,4),stop_words="english")
#vectorizer = TfidfVectorizer()

In [ ]:

similarity = lambda doc1,doc2: cosine_similarity([doc1,doc2])

In [ ]:
vector_train = [line for line in train_contents]
s_vectors = list(zip(train_files,vector_train))

In [ ]:
s_vectors[100]

In [ ]:
# Define the location of the directory
path2 =r"D:\data\Work project\Check Plagiarism\checkdata\testfile"

# Change the directory
os.chdir(path2)

## Check test file

In [ ]:
#Choose the test file
test_name = "suspicious-figure-0_out.txt"

In [ ]:

OLD_PATH = "D:\data\Work project\Check Plagiarism\Dataset\Figure Plagiarism corpus\Shape_Based_Figure Plagiarism\Plagiarised figures\Exact copy"

test_file = read_files(os.path.join(OLD_PATH,test_name))

In [ ]:
test_file

In [ ]:
train_files[100]

In [ ]:
import numpy as np

In [ ]:
def write_result(test_file):
    result = dict()
    for i in range(len(test_file)):
        result["line"+str(i)] = dict()
        result["line"+str(i)]["name"] = []
        result["line"+str(i)]["point"] = []
    return result

In [ ]:
def check_plagiarism(s_vectors,test_file):
    test_result= {}
    for test_line in range(len(test_file)):
        test_result["line"+str(test_line)] =dict()
        for a, vector_a in s_vectors:
            test_result["line"+str(test_line)][a] = dict()
            for train_line in range(len(vector_a)):
                documents = [vector_a[train_line],test_file[test_line]]
                sparse_matrix = vectorizer.fit_transform(documents)
                score = cosine_similarity(sparse_matrix)[0][1]
                test_result["line"+str(test_line)][a]["line"+str(train_line)] = score
    return test_result

In [ ]:
test_result = check_plagiarism(s_vectors,test_file)

In [ ]:
for test_line in range(len(test_file)):
    test_result["line"+str(test_line)] =dict()
    for a, vector_a in s_vectors:
        #print(a)
        test_result["line"+str(test_line)][a] = dict()

In [ ]:
#test_result

## Show the result

In [ ]:
import pandas as pd
dt = pd.DataFrame(test_result["line0"])

In [ ]:
def get_column_name_for_max_values_of(df):
    A = df.columns[df.isin([max(df.T.max())]).any()].item() 
    B = df.T.columns[df.T.isin([max(df.max())]).any()].item()
    return A, B, max(df.max())

In [ ]:
get_column_name_for_max_values_of(dt)

## Check plagiarism with not exact data

In [ ]:
OLD_PATH2 = "D:\data\Work project\Check Plagiarism\Dataset\Figure Plagiarism corpus\Shape_Based_Figure Plagiarism\Plagiarised figures\Text plagiarism"
test_name2 = "Suspicious-figure-275_out.txt"
test_file2 = read_files(os.path.join(OLD_PATH2,test_name2))

In [ ]:
#test_file2

In [ ]:
result2 = check_plagiarism(s_vectors,test_file2)

In [ ]:
#result2

In [ ]:
dt2 = pd.DataFrame(result2["line0"])

In [ ]:
get_column_name_for_max_values_of(dt2)

## Second method

In [ ]:
# import gensim

# from gensim import corpora
# import gensim.downloader as api
# from gensim.utils import simple_preprocess
# print(gensim.__version__)


# # Download the FastText model
# fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

In [ ]:
def check_plagiarism_soft(s_vectors,test_file):
    test_result= {}
    for test_line in range(len(test_file)):
        test_result["line"+str(test_line)] =dict()
        for a, vector_a in s_vectors:
            test_result["line"+str(test_line)][a] = dict()
            for train_line in range(len(vector_a)):
#                 lemmatized_trainline = [wordnet_lemmatizer.lemmatize(t) for t in vector_a[train_line]]
#                 lemmatized_testline = [wordnet_lemmatizer.lemmatize(t) for t in test_file[test_line]]
                documents = [vector_a[train_line],test_file[test_line]]
                dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])
                similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)
                #sparse_matrix = vectorizer.fit_transform(documents)
                sent_1 = dictionary.doc2bow(simple_preprocess(vector_a[train_line]))
                sent_2 = dictionary.doc2bow(simple_preprocess(test_file[test_line]))
                sentences = [sent_1,sent_2]
                score = cosine_similarity(sentences)[0][1]
                test_result["line"+str(test_line)][a]["line"+str(train_line)] = score
    return test_result

In [ ]:
import tokenize

In [ ]:
from nltk.tokenize import word_tokenize



In [ ]:
a = " test this line with changing some line"
alpha_only = word_tokenize(a)
lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in alpha_only]
lemmatized 


In [ ]:
from gensim.corpora.dictionary import Dictionary


In [ ]:
#result3 = check_plagiarism_soft(s_vectors,test_file2)

In [ ]:
!pip uninstall gensim